# 🎯 LLM Classification Finetuning with PyTorch Lightning

This notebook implements a comprehensive solution for the LLM Classification Finetuning competition using PyTorch Lightning and BERT. The approach involves:

1. **Data Processing**: Loading and preprocessing the training/test data
2. **Model Architecture**: Using BERT with a custom classification head
3. **Training**: Fine-tuning with PyTorch Lightning
4. **Inference**: Generating predictions for the test set

## 📊 Problem Overview

We need to classify which response (Model A, Model B, or Tie) is better for given prompts. This is a 3-class classification problem where we predict probabilities for:
- `winner_model_a`: Probability that Model A's response is better
- `winner_model_b`: Probability that Model B's response is better  
- `winner_tie`: Probability that both responses are equally good


## 📦 Import Libraries and Setup

First, let's import all necessary libraries and set up our environment for reproducible results.


In [ ]:
import warnings
warnings.filterwarnings('ignore', message='.*overflowing tokens.*')

import os
import random
from pathlib import Path
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

print("📚 Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch Lightning version: {pl.__version__}")


## ⚙️ Configuration and Setup

Define all hyperparameters and configuration settings. We're using Kaggle's input directory structure and downloading BERT directly from HuggingFace.


In [ ]:
# Configuration
class Config:
    # Paths - Kaggle format
    BASE_DIR = Path.cwd()
    DATA_PATH = BASE_DIR / '..' / 'input' / 'llm-classification-finetuning'
    
    # Model - Download directly from HuggingFace
    MODEL_NAME = BASE_DIR / '..' / 'input/bert-base-uncased/pytorch/default/1/bert-base-uncased'
    
    # Training parameters
    BATCH_SIZE = 16
    MAX_LENGTH = 512
    LEARNING_RATE = 5e-5
    MAX_EPOCHS = 2
    NUM_WORKERS = 4
    HIDDEN_DIM = 256
    NUM_CLASSES = 3
    DROPOUT_RATE = 0.1
    
    # Other settings
    SEED = 42
    TEST_SIZE = 0.2

def set_seed(seed=42):
    """Set random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Initialize configuration and set seed
config = Config()
set_seed(config.SEED)

print(f"🔧 Configuration initialized")
print(f"📁 Data path: {config.DATA_PATH}")
print(f"🤖 Model: {config.MODEL_NAME}")
print(f"🎲 Random seed: {config.SEED}")


## 📊 Data Loading and Exploration

Load the training data and explore its structure to understand what we're working with.


In [ ]:
def load_data(config):
    """Load and split the training data"""
    print("📊 Loading training data...")
    
    # Load training data
    train_file_path = config.DATA_PATH / 'train.csv'
    if not train_file_path.exists():
        raise FileNotFoundError(f"Training data not found at {train_file_path}")
    
    train_data = pd.read_csv(train_file_path)
    print(f"Training data shape: {train_data.shape}")
    print(f"Training data columns: {train_data.columns.tolist()}")
    
    # Split the data into train and validation sets
    train_data_split, validation_data = train_test_split(
        train_data, 
        test_size=config.TEST_SIZE, 
        random_state=config.SEED, 
        stratify=train_data[['winner_model_a', 'winner_model_b', 'winner_tie']].values
    )
    
    print(f"\n📈 Data split completed:")
    print(f"Training samples: {len(train_data_split)}")
    print(f"Validation samples: {len(validation_data)}")
    
    # Verify label distribution
    print("\n🏷️ Training set label distribution:")
    print(f"Model A wins: {train_data_split['winner_model_a'].sum()}")
    print(f"Model B wins: {train_data_split['winner_model_b'].sum()}")
    print(f"Ties: {train_data_split['winner_tie'].sum()}")
    
    print("\n🏷️ Validation set label distribution:")
    print(f"Model A wins: {validation_data['winner_model_a'].sum()}")
    print(f"Model B wins: {validation_data['winner_model_b'].sum()}")
    print(f"Ties: {validation_data['winner_tie'].sum()}")
    
    return train_data_split, validation_data

# Load and explore the data
train_df, val_df = load_data(config)

# Display sample data
print("\n📋 Sample training data:")
train_df.head(5)


## 🔤 Tokenizer Setup

Initialize the BERT tokenizer that will be used to convert text into tokens that the model can understand.


In [ ]:
# Initialize tokenizer - Download directly from HuggingFace
print(f"🔤 Loading tokenizer: {config.MODEL_NAME}")
tokenizer = BertTokenizer.from_pretrained(config.MODEL_NAME)

print(f"✅ Tokenizer loaded successfully!")
print(f"Vocabulary size: {len(tokenizer)}")
print(f"Max length: {config.MAX_LENGTH}")


## 📚 Dataset Classes

Define custom dataset classes for training and testing data. These classes handle:
- Text preprocessing and formatting
- Label conversion (from one-hot to single class)
- Data structure for efficient batch processing


In [ ]:
class LLMClassificationDataset(Dataset):
    """Dataset for LLM Classification with on-the-fly tokenization"""
    
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        # Prepare the three text inputs
        prompt = row['prompt']
        model_a_response = row['response_a']
        model_b_response = row['response_b']
        
        # Convert labels to single class (0: model_a wins, 1: model_b wins, 2: tie)
        if row['winner_model_a'] == 1:
            label = 0
        elif row['winner_model_b'] == 1:
            label = 1
        else:  # winner_tie == 1
            label = 2
            
        return {
            'prompt': prompt,
            'model_a_response': model_a_response,
            'model_b_response': model_b_response,
            'label': label
        }

class TestDataset(Dataset):
    """Test dataset for inference"""
    
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        # Prepare the three text inputs
        prompt = row['prompt']
        model_a_response = row['response_a']
        model_b_response = row['response_b']
            
        return {
            'prompt': prompt,
            'model_a_response': model_a_response,
            'model_b_response': model_b_response
        }

print("📚 Dataset classes defined successfully!")


## 🔄 Data Collation Functions

Define collate functions that handle batch tokenization. These functions:
- Tokenize text inputs in batches for efficiency
- Handle padding and truncation
- Prepare tensors for model input


In [ ]:
def collate_fn(batch, tokenizer, max_length=512):
    """Custom collate function for batch tokenization"""
    prompts = [item['prompt'] for item in batch]
    model_a_responses = [item['model_a_response'] for item in batch]
    model_b_responses = [item['model_b_response'] for item in batch]
    labels = torch.tensor([item['label'] for item in batch], dtype=torch.long)
    
    # Tokenize each input type
    prompt_encoding = tokenizer(
        prompts, padding=True, truncation=True, 
        max_length=max_length, return_tensors='pt'
    )
    
    model_a_encoding = tokenizer(
        model_a_responses, padding=True, truncation=True,
        max_length=max_length, return_tensors='pt'
    )
    
    model_b_encoding = tokenizer(
        model_b_responses, padding=True, truncation=True,
        max_length=max_length, return_tensors='pt'
    )
    
    return {
        'prompt_input_ids': prompt_encoding['input_ids'],
        'prompt_attention_mask': prompt_encoding['attention_mask'],
        'model_a_input_ids': model_a_encoding['input_ids'],
        'model_a_attention_mask': model_a_encoding['attention_mask'],
        'model_b_input_ids': model_b_encoding['input_ids'],
        'model_b_attention_mask': model_b_encoding['attention_mask'],
        'labels': labels
    }

def test_collate_fn(batch, tokenizer, max_length=512):
    """Custom collate function for test data"""
    prompts = [item['prompt'] for item in batch]
    model_a_responses = [item['model_a_response'] for item in batch]
    model_b_responses = [item['model_b_response'] for item in batch]
    
    # Tokenize each input type
    prompt_encoding = tokenizer(
        prompts, padding=True, truncation=True, 
        max_length=max_length, return_tensors='pt'
    )
    
    model_a_encoding = tokenizer(
        model_a_responses, padding=True, truncation=True,
        max_length=max_length, return_tensors='pt'
    )
    
    model_b_encoding = tokenizer(
        model_b_responses, padding=True, truncation=True,
        max_length=max_length, return_tensors='pt'
    )
    
    return {
        'prompt_input_ids': prompt_encoding['input_ids'],
        'prompt_attention_mask': prompt_encoding['attention_mask'],
        'model_a_input_ids': model_a_encoding['input_ids'],
        'model_a_attention_mask': model_a_encoding['attention_mask'],
        'model_b_input_ids': model_b_encoding['input_ids'],
        'model_b_attention_mask': model_b_encoding['attention_mask']
    }

print("🔄 Collate functions defined successfully!")


## 🧠 Model Architecture

Define the main model architecture using PyTorch Lightning. The model:
- Uses pre-trained BERT as the backbone
- Freezes BERT parameters
- Processes three text inputs: prompt, model_a_response, model_b_response
- Concatenates BERT embeddings and passes through a classification head
- Outputs probabilities for 3 classes


In [ ]:
class LLMClassificationModel(pl.LightningModule):
    """PyTorch Lightning model for LLM Classification"""
    
    def __init__(self, model_name='bert-base-uncased', learning_rate=5e-5, 
                 hidden_dim=256, num_classes=3, dropout_rate=0.1):
        super().__init__()
        self.save_hyperparameters()
        
        # Load BERT model - Download directly from HuggingFace
        print(f"🤖 Loading BERT model: {model_name}")
        self.bert = BertModel.from_pretrained(model_name)
        
        # Freeze most BERT parameters
        for param in self.bert.parameters():
            param.requires_grad = False
            
        # Classification head
        bert_dim = self.bert.config.hidden_size
        self.classifier = nn.Sequential(
            nn.Linear(bert_dim * 3, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, num_classes)
        )
        
        # Loss function
        self.loss_fn = nn.CrossEntropyLoss()
        
    def forward(self, prompt_input_ids, prompt_attention_mask,
                model_a_input_ids, model_a_attention_mask,
                model_b_input_ids, model_b_attention_mask):
        
        # Get embeddings for each input
        prompt_outputs = self.bert(input_ids=prompt_input_ids, attention_mask=prompt_attention_mask)
        prompt_embedding = prompt_outputs.last_hidden_state[:, 0, :]  # CLS token
        
        model_a_outputs = self.bert(input_ids=model_a_input_ids, attention_mask=model_a_attention_mask)
        model_a_embedding = model_a_outputs.last_hidden_state[:, 0, :]  # CLS token
        
        model_b_outputs = self.bert(input_ids=model_b_input_ids, attention_mask=model_b_attention_mask)
        model_b_embedding = model_b_outputs.last_hidden_state[:, 0, :]  # CLS token
        
        # Concatenate embeddings
        concatenated = torch.cat([prompt_embedding, model_a_embedding, model_b_embedding], dim=1)
        
        # Classification
        logits = self.classifier(concatenated)
        return logits
    
    def training_step(self, batch, batch_idx):
        logits = self.forward(
            batch['prompt_input_ids'], batch['prompt_attention_mask'],
            batch['model_a_input_ids'], batch['model_a_attention_mask'],
            batch['model_b_input_ids'], batch['model_b_attention_mask']
        )
        
        loss = self.loss_fn(logits, batch['labels'])
        
        # Calculate accuracy
        preds = torch.argmax(logits, dim=1)
        acc = (preds == batch['labels']).float().mean()
        
        # Logging
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        logits = self.forward(
            batch['prompt_input_ids'], batch['prompt_attention_mask'],
            batch['model_a_input_ids'], batch['model_a_attention_mask'],
            batch['model_b_input_ids'], batch['model_b_attention_mask']
        )
        
        loss = self.loss_fn(logits, batch['labels'])
        
        # Calculate accuracy
        preds = torch.argmax(logits, dim=1)
        acc = (preds == batch['labels']).float().mean()
        
        # Logging
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        self.log('val_acc', acc, on_epoch=True, prog_bar=True)
        
        return {'val_loss': loss, 'preds': preds, 'labels': batch['labels']}
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.learning_rate)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
        return [optimizer], [scheduler]

print("🧠 Model architecture defined successfully!")


## 📊 Data Module

PyTorch Lightning DataModule that handles:
- Dataset initialization
- DataLoader creation with proper batch processing
- Train/validation data splitting


In [ ]:
class LLMDataModule(pl.LightningDataModule):
    """PyTorch Lightning DataModule for LLM Classification"""
    
    def __init__(self, train_df, val_df, tokenizer, batch_size=16, max_length=512, num_workers=4):
        super().__init__()
        self.train_df = train_df
        self.val_df = val_df
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_length = max_length
        self.num_workers = num_workers
        
    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            self.train_dataset = LLMClassificationDataset(self.train_df, self.tokenizer, self.max_length)
            self.val_dataset = LLMClassificationDataset(self.val_df, self.tokenizer, self.max_length)
    
    def _collate_fn(self, batch):
        """Wrapper for collate_fn that can be pickled"""
        return collate_fn(batch, self.tokenizer, self.max_length)
    
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=4,  
            collate_fn=self._collate_fn,
            pin_memory=False
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=4,  
            collate_fn=self._collate_fn,
            pin_memory=False
        )

print("📊 Data module defined successfully!")


## 🚀 Model Training

Now let's train our model using PyTorch Lightning. The training includes:
- Model checkpointing (saves best models)
- Early stopping (prevents overfitting)
- Mixed precision training (for efficiency)
- Validation once per epoch (as requested)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
# Initialize data module
data_module = LLMDataModule(
    train_df=train_df,
    val_df=val_df,
    tokenizer=tokenizer,
    batch_size=config.BATCH_SIZE,
    max_length=config.MAX_LENGTH,
    num_workers=config.NUM_WORKERS
)

# Initialize model
model = LLMClassificationModel(
    model_name=config.MODEL_NAME,
    learning_rate=config.LEARNING_RATE,
    hidden_dim=config.HIDDEN_DIM,
    num_classes=config.NUM_CLASSES,
    dropout_rate=config.DROPOUT_RATE
)

# Set up callbacks
checkpoint_callback = ModelCheckpoint(
    monitor='val_acc',
    dirpath='./checkpoints',
    filename='llm-classification-{epoch:02d}-{val_acc:.2f}',
    save_top_k=3,
    mode='max'
)

# Set up logger
logger = TensorBoardLogger('tb_logs', name='llm_classification')

# Initialize trainer with mixed precision - VALIDATION ONCE PER EPOCH
trainer = pl.Trainer(
    max_epochs=config.MAX_EPOCHS,
    precision='16-mixed',  # Enable mixed precision training
    accelerator='auto',    # Automatically detect GPU/CPU
    devices='auto',        # Use all available devices
    callbacks=[checkpoint_callback],
    logger=logger,
    log_every_n_steps=50,
    val_check_interval=1.0,  # Validate once per epoch
    gradient_clip_val=1.0,   # Gradient clipping
    accumulate_grad_batches=1,
    deterministic=True
)

print("🚀 Starting training with PyTorch Lightning...")
print(f"Using device: {trainer.strategy.root_device}")
print(f"Mixed precision: {'Enabled' if trainer.precision == '16-mixed' else 'Disabled'}")
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Validation frequency: Once per epoch")

# Train the model
trainer.fit(model, data_module)

print("✅ Training completed!")
print(f"Best model saved at: {checkpoint_callback.best_model_path}")
print(f"Best validation accuracy: {checkpoint_callback.best_model_score:.4f}")


## 🔍 Test Data Loading

Load the test data for inference and prepare it for prediction.


In [ ]:
def load_test_data(config):
    """Load test data for inference"""
    test_file_path = config.DATA_PATH / 'test.csv'
    if not test_file_path.exists():
        raise FileNotFoundError(f"Test data not found at {test_file_path}")
    
    test_data = pd.read_csv(test_file_path)
    print(f"Test data shape: {test_data.shape}")
    print(f"Test data columns: {test_data.columns.tolist()}")
    
    return test_data

# Load test data
print("🔍 Loading test data...")
test_data = load_test_data(config)

print("\n📋 Sample test data:")
test_data.head(2)


## 🎯 Inference and Submission

Generate predictions on the test set using our trained model and create the submission file. The output will be saved as `submission.csv` with probabilities for each class.


In [ ]:
def run_inference(test_data, checkpoint_path, tokenizer, config):
    """Run inference on test data"""
    print("🔍 Starting inference...")
    
    # Load the best model checkpoint
    best_model = LLMClassificationModel.load_from_checkpoint(checkpoint_path)
    best_model.eval()
    best_model.freeze()
    
    # Create test dataset and dataloader
    test_dataset = TestDataset(test_data, tokenizer, max_length=config.MAX_LENGTH)
    
    def _test_collate_fn(batch):
        """Wrapper for test_collate_fn that can be pickled"""
        return test_collate_fn(batch, tokenizer, max_length=config.MAX_LENGTH)
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=32,
        shuffle=False,
        num_workers=4, 
        collate_fn=_test_collate_fn,
        pin_memory=False
    )
    
    # Perform inference
    print("🔍 Running inference on test data...")
    all_predictions = []
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(test_loader):
            # Move batch to device
            batch = {k: v.to(best_model.device) for k, v in batch.items()}
            
            # Get logits
            logits = best_model(
                batch['prompt_input_ids'], batch['prompt_attention_mask'],
                batch['model_a_input_ids'], batch['model_a_attention_mask'],
                batch['model_b_input_ids'], batch['model_b_attention_mask']
            )
            
            # Apply softmax to get probabilities
            probabilities = F.softmax(logits, dim=-1)
            all_predictions.append(probabilities.cpu())
            
            if (batch_idx + 1) % 10 == 0:
                print(f"Processed {batch_idx + 1}/{len(test_loader)} batches")
    
    # Concatenate all predictions
    all_predictions = torch.cat(all_predictions, dim=0)
    
    # Extract probabilities for each class
    winner_model_a_probs = all_predictions[:, 0].numpy()  # Class 0: model_a wins
    winner_model_b_probs = all_predictions[:, 1].numpy()  # Class 1: model_b wins
    winner_tie_probs = all_predictions[:, 2].numpy()      # Class 2: tie
    
    # Create submission dataframe
    submission = pd.DataFrame({
        'id': test_data['id'],
        'winner_model_a': winner_model_a_probs,
        'winner_model_b': winner_model_b_probs,
        'winner_tie': winner_tie_probs
    })
    
    # Verify probabilities sum to 1
    prob_sums = submission[['winner_model_a', 'winner_model_b', 'winner_tie']].sum(axis=1)
    print(f"Probability sums - Min: {prob_sums.min():.4f}, Max: {prob_sums.max():.4f}, Mean: {prob_sums.mean():.4f}")
    
    print("📊 Submission preview:")
    print(submission.head())
    
    # Save submission as requested filename
    submission.to_csv('submission.csv', index=False)
    print("✅ Submission file saved as 'submission.csv'")
    
    return submission

# Run inference and create submission
submission = run_inference(test_data, checkpoint_callback.best_model_path, tokenizer, config)

print("\n🎉 Pipeline completed successfully!")
print(f"📁 Best model: {checkpoint_callback.best_model_path}")
print(f"📄 Submission: submission.csv")
print(f"🎯 Final validation accuracy: {checkpoint_callback.best_model_score:.4f}")


## 📈 Results Summary

Let's review our final results and submission statistics.


In [ ]:
# Display submission statistics
print("📈 Final Submission Statistics:")
print(f"Total test samples: {len(submission)}")
print("\n📊 Probability distributions:")
print(f"Model A wins - Mean: {submission['winner_model_a'].mean():.4f}, Std: {submission['winner_model_a'].std():.4f}")
print(f"Model B wins - Mean: {submission['winner_model_b'].mean():.4f}, Std: {submission['winner_model_b'].std():.4f}")
print(f"Ties - Mean: {submission['winner_tie'].mean():.4f}, Std: {submission['winner_tie'].std():.4f}")

# Check for any potential issues
print("\n🔍 Data validation:")
print(f"Any NaN values: {submission.isnull().any().any()}")
print(f"All probabilities >= 0: {(submission[['winner_model_a', 'winner_model_b', 'winner_tie']] >= 0).all().all()}")
print(f"All probabilities <= 1: {(submission[['winner_model_a', 'winner_model_b', 'winner_tie']] <= 1).all().all()}")

print("\n✅ Submission file 'submission.csv' is ready for upload!")
